In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
%matplotlib inline

from transplant.config import *

In [2]:
df_static = pd.read_csv(PATH_STATIC_CLEAN)
df_dynamic =  pd.read_csv(PATH_DYNAMIC_CLEAN)

## General functions

In [3]:
def time_offset_dynamic(df, time_offset=60):
    """
    Function that truncate the dynamic dataset to take into account only the last minutes (time_offset) of the operation
    """
    df['time']= pd.to_datetime(df['time'])
    date_min = df['time'].max() - timedelta(minutes=time_offset)
    return df[df['time'] >= date_min]

In [4]:
def Sufficient_decline(declampage_2,fermeture, decline=1/3, epsilon = 0.001):
    """
    Function that test if the decline of a value between the event 'declampage_cote_2' and 'fermeture' is sufficient
    decline : fraction of the value at time 'declampage_2' 
            (ie "on veut que la noredraline baisse d'au moins 1/3 de sa valeur")
    epsilon : "nearly equal"
    """
    if fermeture > declampage_2:
        return False
    delta = declampage_2 - fermeture
    if delta > decline*declampage_2:
        return True
    return np.abs(delta - decline*declampage_2)<= epsilon

## Test functions

In [5]:
def test_PASm_sup(data_dyn, threshold=90, obj=60):
    """
    Test if threshold% of our value (PASm) is > objective /do not take into account PASm = 0
    Return boolean
    True : "at the end of surgery, when PASm not equal 0, at least 90% of the values are > 60"
    """
    df = time_offset_dynamic(data_dyn)
    non_zeros = df['PASm'][df['PASm']!=0].count()
    nb_ok = df['PASm'][df['PASm']>obj].count()
    return (nb_ok/non_zeros)*100>threshold

In [6]:
def test_noradrenaline(data,nora_max=1.5, nora_min=1):
    """
    Test if the noradrenaline is sufficiently low :
        > 1.5mg/h --> ko
        < 1mh/h --> ok
        between : see if there is a sufficient decline
    """
    if data['NORAD_fermeture'].values[0] >= nora_max:
        return False
    elif data['NORAD_fermeture'].values[0] <= nora_min:
        return True
    else : 
        return Sufficient_decline(data['NORAD_declampage_cote_2'].values[0],
                                  data['NORAD_fermeture'].values[0],
                                  decline=1/3, epsilon = 0.001)

In [7]:
def test_ratio_pao2_fio2(data,obj):
    """
    Verify if the ratio PaO2/FiO2 ratio is high enough
    True : PaO2/FiO2 ratio > obj
    """
    ratio = data['PaO2_fermeture'].values[0] / data['FiO2_fermeture'].values[0]
    return ratio >= obj

In [8]:
def test_PAP_diminution(data, paps_max=50, paps_min=30):
    """
    True : If PAPS, PAPM, PAPD decrease 
    /!\ : lot of nan for  PAPm -> take PAPs
    if nan return False
    """
    if data['PAPS_fermeture'].values[0] < paps_min:
        return True
    elif data['PAPS_fermeture'].values[0] >= paps_max:
        return False
    else :
        return Sufficient_decline(data['PAPS_declampage_cote_2'].values[0],
                                  data['PAPS_fermeture'].values[0],
                                  decline=1/3, epsilon = 0.001)

In [9]:
def test_augm_ratio_pao2_fio2(data,threshold_augm=0):
    """
    Test if ratio sufficiently high or sufficient evolution between the events 'declapage_cote_2' & 'fermeture'
    """
    if test_ratio_pao2_fio2(data,obj=300):
        return True
    ratio_fermeture = data['PaO2_fermeture'].values[0] / data['FiO2_fermeture'].values[0]
    ratio_declampage_cote_2 = data['PaO2_declampage_cote_2'].values[0] / data['FiO2_declampage_cote2'].values[0]
    return ratio_fermeture - ratio_declampage_cote_2 > threshold_augm

In [10]:
def test_temperature_sup36(data_dyn,threshold=90):
    """
    Test if threshold% of our value (Temp) is > 36° /do not take into account Temp = 0
    Return boolean
    """
    df = time_offset_dynamic(data_dyn)
    non_zeros = df['Temp'][df['Temp']!=0].count()
    nb_ok = df['Temp'][df['Temp']>36].count()
    return (nb_ok/non_zeros)*100>threshold

In [11]:
def test_NMT_sup(data_dyn, obj=50, threshold=90): #threshold ok ou trop severe ?
    """
    Test if threshold% of our value (NMTratio) is > 50% /do not take into account Temp = 0
    """
    df = time_offset_dynamic(data_dyn)
    non_zeros = df['NMTratio'][df['NMTratio']!=0].count()
    nb_ok = df['NMTratio'][df['NMTratio']>obj].count() 
    return (nb_ok/non_zeros)*100>=threshold

## Split functions

In [117]:
def split_ECMO_during_surgery(data):
    """
    Return boolean 
    True : presence of ECMO during surgery
    False : absence of ECMO during surgery
    """
    if data['ECMO_during_surgery'].values[0]==1:
        return "Node ECMO weaning Test"
    return "Node ratio-100 test"

In [118]:
def split_ECMO_weaning_test(data, data_dyn):
    """
    Return boolean 
    """
    sum_True = int(test_PASm_sup(data_dyn, threshold=90, obj=60)) + \
               int(test_noradrenaline(data, nora_max=1.5, nora_min=1)) + \
               int(test_ratio_pao2_fio2(data, obj=200)) + \
               int(test_PAP_diminution(data, paps_max=50, paps_min=30))
    if sum_True == 4:
        return "Node INO weaning Test"
    return "Transfer to ICU with ECMO and mechanical ventilation"

In [119]:
def split_ratio_100(data):
    if test_ratio_pao2_fio2(data, obj=100) :
        return "Node INO weaning Test"
    return "Transfer to ICU with ECMO and mechanical ventilation"

In [120]:
def split_INO_weaning_test(data):
    """
    Return boolean 
    """
    sum_True = int(test_augm_ratio_pao2_fio2(data,threshold_augm=0)) + \
               int(test_PAP_diminution(data, paps_max=50, paps_min=30))
    if sum_True == 2:
        return "Node ratio-300 test"
    return "Transfer to ICU with mechanical ventilation and iNO therapy"

In [121]:
def split_ratio_300(data):
    if test_ratio_pao2_fio2(data, obj=300) :
        return "Node NIV attempt"
    return "Transfer to ICU with mechanical ventilation"

In [122]:
def split_NIV_attempt(data, data_dyn):
    """
    Return boolean 
    """
    sum_True = int(test_temperature_sup36(data_dyn,threshold=90)) + \
               int(test_NMT_sup(data_dyn,obj=50, threshold=90))+ \
               int(test_PASm_sup(data_dyn, threshold=80, obj=60)) + \
               int(test_noradrenaline(data, nora_max=1.5, nora_min=1)) + \
               int(test_ratio_pao2_fio2(data,obj=300))
    if sum_True ==5:
        return  "Transfer to ICU with standard oxygen therapy or NIV"
    return "Transfer to ICU with mechanical ventilation"

## Validation Tree

In [125]:
def decision_empirique(df_static,df_dynamic,id_patient):
     
    df_static_temp = df_static[df_static['id_patient']==id_patient]
    df_dynamic_temp = df_dynamic[df_dynamic['id_patient']==id_patient]
    
                               
    if split_ECMO_during_surgery(df_static_temp) == "Node ECMO weaning Test":
        if split_ECMO_weaning_test(df_static_temp, df_dynamic_temp) == "Node INO weaning Test":
            if split_INO_weaning_test(df_static_temp) == "Node ratio-300 test":
                if split_ratio_300(df_static_temp) == "Node NIV attempt":
                    return split_NIV_attempt(df_static_temp, df_dynamic_temp)
                else:
                    return split_ratio_300(df_static_temp)
            else : 
                return split_INO_weaning_test(df_static_temp)
        else :
            return split_ECMO_weaning_test(df_static_temp, df_dynamic_temp)
        
    else: #Node ratio-100 test"
        if split_ratio_100(df_static_temp) =="Node INO weaning Test":
            if split_INO_weaning_test(df_static_temp) == "Node ratio-300 test":
                if split_ratio_300(df_static_temp) == "Node NIV attempt":
                    return split_NIV_attempt(df_static_temp, df_dynamic_temp)
                else:
                    return split_ratio_300(df_static_temp)
            else : 
                return split_INO_weaning_test(df_static_temp)
        else : 
            return split_ratio_100(df_static_temp)

## Tests

- avec les données postops
- avec notre label à nous

In [126]:
def recode_leaf_in_immediate_extubation(leaf):
    """ recode the leaves of the tree with immediate_extubation 0/1"""
    if leaf =="Transfer to ICU with standard oxygen therapy or NIV":
        return 1
    return 0

In [127]:
df_results=df_static[['id_patient','immediate_extubation']]

In [128]:
ids = df_results['id_patient'].values.tolist()
leaves = [decision_empirique(df_static,df_dynamic,id_patient=x) for x in ids]
recode = [recode_leaf_in_immediate_extubation(x) for x in leaves]

/home/karine/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/karine/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/home/karine/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  
/home/karine/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [129]:
df_results['leaves']=leaves
df_results['leaves_1/0']=recode

/home/karine/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/karine/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#add true 

In [43]:
from transplant.data.dataset import Dataset

In [44]:
dataset = Dataset()
train, test = dataset.get_static()
data_static = pd.concat([train, test])

In [132]:
df_results = pd.merge(df_results, data_static[['id_patient','target']], how='left', on='id_patient')

In [133]:
df_results.head()

,id_patient,immediate_extubation,leaves,leaves_1/0,target
0,1,0,Transfer to ICU with mechanical ventilation,0,0
1,2,1,Transfer to ICU with standard oxygen therapy o...,1,1
2,3,0,Transfer to ICU with ECMO and mechanical venti...,0,0
3,4,0,Transfer to ICU with mechanical ventilation an...,0,1
4,5,0,Transfer to ICU with mechanical ventilation an...,0,0


In [134]:
df_results[["immediate_extubation","leaves","leaves_1/0",'target']].groupby('leaves').sum()

,immediate_extubation,leaves_1/0,target
leaves,,,
Transfer to ICU with ECMO and mechanical ventilation,27,0,41
Transfer to ICU with mechanical ventilation,22,0,26
Transfer to ICU with mechanical ventilation and iNO therapy,82,0,95
Transfer to ICU with standard oxygen therapy or NIV,6,7,7
